# https://github.com/nsidc/NSIDC-Data-Tutorials/blob/main/notebooks/SnowEx_ASO_MODIS_Snow/Snow-tutorial.ipynb

In [5]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from os.path import join, expanduser, basename
import os
import geopandas as gpd
import rasterio as rio
from shapely.geometry import box
from shapely.geometry.polygon import orient
import matplotlib as mpl
from rasterio.features import dataset_features
from shapely.ops import unary_union
from shapely.geometry import Point, Polygon
import folium
from folium import plugins

from invert import invert_sd, epsilon_density
import nsidc_helpers as fn
from folium_vis import add_ee_layer
folium.Map.add_ee_layer = add_ee_layer


In [15]:
def rio_to_exterior(fp, simplify = False):
    with rio.open(fp) as src:
        shapes = list(dataset_features(src, bidx=1, as_mask=False, geographic=True, band=False))
        result = gpd.GeoDataFrame.from_dict(shapes, crs = 'EPSG:4326')
        for i in result.index:
            result.iloc[i]['geometry'] = Polygon(result.iloc[i]['geometry']['coordinates'][0])
        boundary = gpd.GeoSeries(unary_union(result['geometry']))
        boundary_gdf = gpd.GeoDataFrame(boundary, columns = ['geometry'], crs = 'EPSG:4326')

    if simplify:
        boundary_gdf = orient(boundary_gdf.simplify(0.01, preserve_topology=False).loc[0],sign=1.0)
    return boundary_gdf

In [17]:
with open(expanduser('~/scratch/data/uavsar/image_fps'), 'rb') as f:
    image_fps = pickle.load(f)

for i in image_fps:
    if i['fp'] == '/bsuscratch/zacharykeskinen/data/uavsar/images/Salt Lake City, UT/stlake_27129_21010-001_21011-012_0007d_s01_L090_01_int_grd/stlake_27129_21010-001_21011-012_0007d_s01_L090HH_01.unw.grd.tiff':
        if basename(i['fp']).split('_')[-2][-2:] == 'HH':
                fp = i['fp']
                with rio.open(fp) as src:
                    bounds = src.bounds
                df = pd.read_csv(i['ann'], index_col = [0])
                s = pd.to_datetime(df.loc['value','start time of acquisition for pass 1']).tz_localize(None)
                e = pd.to_datetime(df.loc['value','start time of acquisition for pass 2']).tz_localize(None)
                cor = i['cor']
                b_gdf = rio_to_exterior(i['fp'], simplify = True)

/bsuhome/zacharykeskinen/miniconda3/envs/proj/lib/python3.9/site-packages/geopandas/geodataframe.py:475: FutureWarning: CRS mismatch between CRS of the passed geometries and 'crs'. Use 'GeoDataFrame.set_crs(crs, allow_override=True)' to overwrite CRS or 'GeoDataFrame.to_crs(crs)' to reproject geometries. CRS mismatch will raise an error in the future versions of GeoPandas.
  return GeoDataFrame(dataframe, geometry=geometry, crs=crs)


In [23]:
polygon = ','.join([str(c) for xy in zip(*poly.exterior.coords.xy) for c in xy])

In [41]:
s.to_datetime64()

numpy.datetime64('2021-02-03T23:10:23.000000000')

In [58]:
a = s.strftime('%Y-%m-%dT%H:%M:%SZ')
b = e.strftime('%Y-%m-%dT%H:%M:%SZ')
temporal = f'{a},{b}'

In [59]:
temporal

'2021-02-03T23:10:23Z,2021-02-10T20:03:35Z'

In [60]:
data_dict = {
             'modis': {'short_name': 'MOD10A1','version': '6','polygon': polygon,'temporal':temporal}
            }

In [62]:
for k, v in data_dict.items(): fn.granule_info(data_dict[k])

There are 8 files of MOD10A1 version 6 over my area and time of interest.
The average size of each file is 7.32 MB and the total size of all 8 granules is 58.57 MB


In [63]:
search_df = fn.time_overlap(data_dict)
print(len(search_df), ' total files returned')
search_df

8  total files returned


,dataset_id,short_name,version,producer_granule_id,start_date,end_date
0,MODIS/Terra Snow Cover Daily L3 Global 500m SI...,MOD10A1,006,MOD10A1.A2021034.h09v04.006.2021037024759.hdf,2021-02-03T00:00:00.000Z,2021-02-03T23:59:59.000Z
1,MODIS/Terra Snow Cover Daily L3 Global 500m SI...,MOD10A1,006,MOD10A1.A2021035.h09v04.006.2021037041504.hdf,2021-02-04T00:00:00.000Z,2021-02-04T23:59:59.000Z
2,MODIS/Terra Snow Cover Daily L3 Global 500m SI...,MOD10A1,006,MOD10A1.A2021036.h09v04.006.2021038043512.hdf,2021-02-05T00:00:00.000Z,2021-02-05T23:59:59.000Z
3,MODIS/Terra Snow Cover Daily L3 Global 500m SI...,MOD10A1,006,MOD10A1.A2021037.h09v04.006.2021039234913.hdf,2021-02-06T00:00:00.000Z,2021-02-06T23:59:59.000Z
4,MODIS/Terra Snow Cover Daily L3 Global 500m SI...,MOD10A1,006,MOD10A1.A2021038.h09v04.006.2021040032829.hdf,2021-02-07T00:00:00.000Z,2021-02-07T23:59:59.000Z
5,MODIS/Terra Snow Cover Daily L3 Global 500m SI...,MOD10A1,006,MOD10A1.A2021039.h09v04.006.2021041035621.hdf,2021-02-08T00:00:00.000Z,2021-02-08T23:59:59.000Z
6,MODIS/Terra Snow Cover Daily L3 Global 500m SI...,MOD10A1,006,MOD10A1.A2021040.h09v04.006.2021042032856.hdf,2021-02-09T00:00:00.000Z,2021-02-09T23:59:59.000Z
7,MODIS/Terra Snow Cover Daily L3 Global 500m SI...,MOD10A1,006,MOD10A1.A2021041.h09v04.006.2021043034533.hdf,2021-02-10T00:00:00.000Z,2021-02-10T23:59:59.000Z


In [64]:
# Create new dictionary with fields needed for CMR url search

url_df = search_df.drop(columns=['start_date', 'end_date','version','dataset_id'])
url_dict = url_df.to_dict('records')

# CMR search variables
granule_search_url = 'https://cmr.earthdata.nasa.gov/search/granules'
headers= {'Accept': 'application/json'}

# Create URL list from each df row
urls = []
for i in range(len(url_dict)):
    response = requests.get(granule_search_url, params=url_dict[i], headers=headers)
    results = json.loads(response.content)
    urls.append(fn.cmr_filter_urls(results))
# flatten url list
urls = list(np.concatenate(urls))
urls

['https://n5eil01u.ecs.nsidc.org/DP4/MOST/MOD10A1.006/2021.02.03/MOD10A1.A2021034.h09v04.006.2021037024759.hdf',
 'https://n5eil01u.ecs.nsidc.org/DP4/MOST/MOD10A1.006/2021.02.03/MOD10A1.A2021034.h09v04.006.2021037024759.hdf.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/MOST/MOD10A1.006/2021.02.04/MOD10A1.A2021035.h09v04.006.2021037041504.hdf',
 'https://n5eil01u.ecs.nsidc.org/DP4/MOST/MOD10A1.006/2021.02.04/MOD10A1.A2021035.h09v04.006.2021037041504.hdf.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/MOST/MOD10A1.006/2021.02.05/MOD10A1.A2021036.h09v04.006.2021038043512.hdf',
 'https://n5eil01u.ecs.nsidc.org/DP4/MOST/MOD10A1.006/2021.02.05/MOD10A1.A2021036.h09v04.006.2021038043512.hdf.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/MOST/MOD10A1.006/2021.02.06/MOD10A1.A2021037.h09v04.006.2021039234913.hdf',
 'https://n5eil01u.ecs.nsidc.org/DP4/MOST/MOD10A1.006/2021.02.06/MOD10A1.A2021037.h09v04.006.2021039234913.hdf.xml',
 'https://n5eil01u.ecs.nsidc.org/DP4/MOST/MOD10A1.006/2021.02.07/MOD10A1.A202103

In [65]:
bounds = poly.bounds # Get polygon bounds to be used as bounding box input
data_dict['modis']['bbox'] = ','.join(map(str, list(bounds))) # Add bounding box subsetting to MODIS dictionary
data_dict['modis']['format'] = 'GeoTIFF' # Add geotiff reformatting to MODIS dictionary

# Set new temporal range based on dataframe above. Note that this will request all MOD10A1 data falling within this time range.
modis_start = min(search_df.loc[search_df['short_name'] == 'MOD10A1', 'start_date'])
modis_end = max(search_df.loc[search_df['short_name'] == 'MOD10A1', 'end_date'])
data_dict['modis']['temporal'] = ','.join([modis_start,modis_end])
print(data_dict['modis'])

{'short_name': 'MOD10A1', 'version': '6', 'polygon': '-111.81438888,40.71720156,-111.81744468000001,40.52118588,-111.75460632000001,40.5176856,-111.73877172,40.52446392,-111.7472724,40.54279872,-111.72410388,40.5510216,-111.67082184,40.53818724,-111.65104248,40.55146608,-111.66026544,40.569356400000004,-111.63915264,40.569356400000004,-111.64159728,40.589358,-111.71432532,40.56752292,-111.75738432,40.5788016,-111.7167144,40.59402504,-111.7553286,40.60052556,-111.71965908,40.607415,-111.69182352,40.58874684,-111.6161508,40.6001922,-111.60676116,40.58502432,-111.59348232,40.59108036,-111.59564916000001,40.57702368,-111.54647856,40.5765792,-111.59270448,40.57202328,-111.58037016,40.56030012,-111.59353788,40.56446712,-111.59042652000001,40.54340988,-111.63737472,40.55535528,-111.60098292,40.52979768,-111.63704136,40.543521,-111.61009476,40.5232416,-111.62798508,40.51707444,-111.43758096,40.51674108,-111.43552524,40.69897788,-111.6122616,40.687977000000004,-111.77944164,40.70181144,-111.814

In [66]:
base_url = 'https://n5eil02u.ecs.nsidc.org/egi/request' # Set NSIDC data access base URL
#data_dict['modis']['request_mode'] = 'stream' # Set the request mode to asynchronous

param_string = '&'.join("{!s}={!r}".format(k,v) for (k,v) in data_dict['modis'].items()) # Convert param_dict to string
param_string = param_string.replace("'","") # Remove quotes

api_request = [f'{base_url}?{param_string}']
print(api_request[0]) # Print API base URL + request parameters

https://n5eil02u.ecs.nsidc.org/egi/request?short_name=MOD10A1&version=6&polygon=-111.81438888,40.71720156,-111.81744468000001,40.52118588,-111.75460632000001,40.5176856,-111.73877172,40.52446392,-111.7472724,40.54279872,-111.72410388,40.5510216,-111.67082184,40.53818724,-111.65104248,40.55146608,-111.66026544,40.569356400000004,-111.63915264,40.569356400000004,-111.64159728,40.589358,-111.71432532,40.56752292,-111.75738432,40.5788016,-111.7167144,40.59402504,-111.7553286,40.60052556,-111.71965908,40.607415,-111.69182352,40.58874684,-111.6161508,40.6001922,-111.60676116,40.58502432,-111.59348232,40.59108036,-111.59564916000001,40.57702368,-111.54647856,40.5765792,-111.59270448,40.57202328,-111.58037016,40.56030012,-111.59353788,40.56446712,-111.59042652000001,40.54340988,-111.63737472,40.55535528,-111.60098292,40.52979768,-111.63704136,40.543521,-111.61009476,40.5232416,-111.62798508,40.51707444,-111.43758096,40.51674108,-111.43552524,40.69897788,-111.6122616,40.687977000000004,-111.779